In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df_events = (
    spark.read.option("header", "true").csv("/home/iceberg/data/events.csv")
)

df_devices = (
    spark.read.option("header", "true").csv("/home/iceberg/data/devices.csv")
    .select(
        "browser_type",
        "os_type",
        "device_type",
        "device_id"
    )
    .withColumnRenamed("browser_type", "browser_family")
    .withColumnRenamed("os_type", "os_family")
    .withColumnRenamed("device_type", "device_family")
)

df = (
    df_events
    .join(df_devices, ["device_id"], "inner")
    .withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))
)

df.show(10)

+---------+-----------+--------+--------------------+---+--------------------+--------------+---------+-------------+-------------------+
|device_id|    user_id|referrer|                host|url|          event_time|browser_family|os_family|device_family|         event_date|
+---------+-----------+--------+--------------------+---+--------------------+--------------+---------+-------------+-------------------+
|532630305| 1037710827|    NULL| www.zachwilson.tech|  /|2021-03-08 17:27:...|         Other|    Other|        Other|2021-03-08 00:00:00|
|532630305|  925588856|    NULL|    www.eczachly.com|  /|2021-05-10 11:26:...|         Other|    Other|        Other|2021-05-10 00:00:00|
|532630305|-1180485268|    NULL|admin.zachwilson....|  /|2021-02-17 16:19:...|         Other|    Other|        Other|2021-02-17 00:00:00|
|532630305|-1044833855|    NULL| www.zachwilson.tech|  /|2021-09-24 15:53:...|         Other|    Other|        Other|2021-09-24 00:00:00|
|532630305|  747494706|    NULL| w

In [15]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host"), col("browser_family")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

sorted.show(10)

+-----------+-----------+--------+--------------------+--------------------+--------------------+--------------+---------+------------------+-------------------+
|  device_id|    user_id|referrer|                host|                 url|          event_time|browser_family|os_family|     device_family|         event_date|
+-----------+-----------+--------+--------------------+--------------------+--------------------+--------------+---------+------------------+-------------------+
|  532630305| 1129583063|    NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|         Other|    Other|             Other|2021-01-07 00:00:00|
|  532630305|-1180485268|    NULL|    www.eczachly.com|                   /|2021-01-07 18:45:...|         Other|    Other|             Other|2021-01-07 00:00:00|
|  532630305| 1129583063|    NULL|    www.eczachly.com|                   /|2021-01-07 21:57:...|         Other|    Other|             Other|2021-01-07 00:00:00|
| 1088283544| -648945006|   

In [11]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [12]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [13]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);


++
||
++
++

In [16]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);

++
||
++
++

In [17]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);

++
||
++
++

In [18]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_family"), col("host"))

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [19]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files
    
UNION ALL
    
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files

size,num_files,sorted
5091394,4,sorted
5553012,4,unsorted


In [21]:
%%sql

SELECT *
FROM demo.bootcamp.events_sorted.files

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/bootcamp/events_sorted/data/00000-72-c93bb32e-4418-4f27-8a77-1806ea9558b7-0-00001.parquet,PARQUET,1,Row(event_date=None),89391,1031403,"{1: 107448, 2: 61005, 3: 11437, 4: 12908, 5: 10692, 6: 7365, 7: 426434, 8: 2274, 9: 77406, 10: 310046}","{1: 89391, 2: 89391, 3: 89391, 4: 89391, 5: 89391, 6: 89391, 7: 89391, 8: 89391, 9: 89391, 10: 89391}","{1: 0, 2: 46359, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1}",{},"{1: bytearray(b'/'), 2: bytearray(b'52.20.78.240'), 3: bytearray(b'%E3%82%A6%E3%82%'), 4: bytearray(b'Android'), 5: bytearray(b'17MB150WB'), 6: bytearray(b'aashish.techcrea'), 7: bytearray(b' \xba\xe7\xb8\xa8\xb8\x05\x00'), 8: bytearray(b'\x00\xa0&\xb4\xa8\xb8\x05\x00'), 9: bytearray(b'-100210680'), 10: bytearray(b'-1000095488')}","{1: bytearray(b'/zzageqnf.php?Fp'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 5: bytearray(b'vivo $2'), 6: bytearray(b'zachwilson.techd'), 7: bytearray(b'\xe8\xb0\x1b\x8ec\x03\x06\x00'), 8: bytearray(b'\x00\xe0dqO\x03\x06\x00'), 9: bytearray(b'999535123'), 10: bytearray(b'999884938')}",None,[4],None,0,"Row(browser_family=Row(column_size=11437, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='%E3%82%A6%E3%82%', upper_bound='webprosbot'), device_family=Row(column_size=10692, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='17MB150WB', upper_bound='vivo $2'), device_id=Row(column_size=77406, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='-100210680', upper_bound='999535123'), event_date=Row(column_size=2274, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 0), upper_bound=datetime.datetime(2023, 8, 20, 0, 0)), event_time=Row(column_size=426434, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 1, 19, 764000), upper_bound=datetime.datetime(2023, 8, 20, 23, 59, 41, 89000)), host=Row(column_size=7365, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='aashish.techcrea', upper_bound='zachwilson.techd'), os_family=Row(column_size=12908, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='Android', upper_bound='iOS'), referrer=Row(column_size=61005, value_count=89391, null_value_count=46359, nan_value_count=None, lower_bound='52.20.78.240', upper_bound='zachwilson.tech'), url=Row(column_size=107448, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='/', upper_bound='/zzageqnf.php?Fp'), user_id=Row(column_size=310046, value_count=89391, null_value_count=1, nan_value_count=None, lower_bound='-1000095488', upper_bound='999884938'))"
0,s3://warehouse/bootcamp/events_sorted/data/00001-73-c93bb32e-4418-4f27-8a77-1806ea9558b7-0-00001.parquet,PARQUET,1,Row(event_date=None),99232,1164694,"{1: 142161, 2: 67344, 3: 11896, 4: 16525, 5: 11505, 6: 9100, 7: 475847, 8: 2355, 9: 86496, 10: 336994}","{1: 99232, 2: 99232, 3: 99232, 4: 99232, 5: 99232, 6: 99232, 7: 99232, 8: 99232, 9: 99232, 10: 99232}","{1: 0, 2: 49299, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 58}",{},"{1: bytearray(b'""/?""""<?=print(93'), 2: bytearray(b'""https://www.goo'), 3: bytearray(b') Bot'), 4: bytearray(b'Android'), 5: bytearray(b'13 Pro Max'), 6: bytearray(b'abhishekanand.te'), 7: bytearray(b'(\x83\xb2EX\xb8\x05\x00'), 8: bytearray(b'\x00 \xc9<X\xb8\x05\x00'), 9: bytearray(b'-100210680'), 10: bytearray(b'-1000370060')}","{1: bytearray(b'/zz.php'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 5: bytearray(b'vivo $2'), 6: bytearray(b'zsavi524.techcrf'), 7: bytearray(b'\x88\xb8\x07P;\x03\x06\x00'), 8: bytearray(b""\x00 \xb65\'\x03\x06\x00""), 9: bytearra

In [22]:
%%sql

SELECT *
FROM demo.bootcamp.events_sorted

device_id,user_id,referrer,host,url,event_time,browser_family,os_family,device_family,event_date
589185851,-21136712,None,admin.zachwilson.tech,/,2021-01-12 18:49:28.425000,Chrome,Linux,Other,2021-01-12 00:00:00
589185851,-414920062,None,admin.zachwilson.tech,/,2021-01-12 18:54:30.995000,Chrome,Linux,Other,2021-01-12 00:00:00
589185851,-414920062,None,admin.zachwilson.tech,/,2021-01-12 19:56:56.809000,Chrome,Linux,Other,2021-01-12 00:00:00
589185851,-694958230,None,admin.zachwilson.tech,/,2021-01-12 20:08:15.964000,Chrome,Linux,Other,2021-01-12 00:00:00
-843023486,-2116612468,None,www.eczachly.com,/,2021-01-12 01:13:53.762000,Chrome,Mac OS X,Other,2021-01-12 00:00:00
-843023486,-2116612468,https://www.eczachly.com/,www.eczachly.com,/blog,2021-01-12 01:13:56.914000,Chrome,Mac OS X,Other,2021-01-12 00:00:00
-843023486,-2116612468,https://www.eczachly.com/blog,www.eczachly.com,/graphs,2021-01-12 01:13:58.899000,Chrome,Mac OS X,Other,2021-01-12 00:00:00
-843023486,-2116612468,https://www.eczachly.com/graphs,www.eczachly.com,/graphs,2021-01-12 01:14:01.017000,Chrome,Mac OS X,Other,2021-01-12 00:00:00
-843023486,-2116612468,https://www.eczachly.com/graphs,www.eczachly.com,/about,2021-01-12 01:14:02.615000,Chrome,Mac OS X,Other,2021-01-12 00:00:00
-843023486,-2116612468,https://www.eczachly.com/about,www.eczachly.com,/contact,2021-01-12 01:14:40.193000,Chrome,Mac OS X,Other,2021-01-12 00:00:00


In [90]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
3145713,5


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
